In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set paths for the dataset
train_dir = 'E:\Dataset_chest_xray/train'  # Replace with your dataset path
val_dir = 'E:\Dataset_chest_xray/val'

# Image dimensions and batch size
img_height, img_width = 150, 150
batch_size = 32

# ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and model checkpointing
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('pneumonia_model.h5', save_best_only=True, monitor='val_loss')
]

# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator,
    callbacks=callbacks
)

# Save the model manually (if not using ModelCheckpoint callback)
model.save('pneumonia_model.h5')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load your pre-trained model (ensure the file name and path are correct)
model = load_model('pneumonia_model.h5')

def annotate_image(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if the image was loaded correctly
    if image is None:
        print("Error: Could not load the image. Check the file path.")
        return

    # Resize the image to match the model's expected input size
    try:
        image_resized = cv2.resize(image, (150, 150))  # Adjust size based on your model's input
    except cv2.error as e:
        print("Error resizing image:", e)
        return

    # Convert grayscale image to RGB
    image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)

    # Preprocess the image for the model
    image_array = img_to_array(image_rgb)
    image_array = np.expand_dims(image_array, axis=0) / 255.0  # Normalize

    # Make a prediction
    prediction = model.predict(image_array)
    label = "Pneumonia" if prediction[0][0] > 0.5 else "Normal"

    # Annotate the image based on the label
    color = (0, 0, 255) if label == "Pneumonia" else (0, 255, 0)
    annotated_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.putText(annotated_image, f"Diagnosis: {label}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Highlight pneumonia symptoms if detected
    if label == "Pneumonia":
        # Apply thresholding to detect denser (whiter) regions of the lung
        _, thresholded = cv2.threshold(image, 120, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Draw contours around the detected areas
        if contours:
            cv2.drawContours(annotated_image, contours, -1, (255, 0, 0), 2)  # Blue contours for pneumonia

    # Display the annotated image
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
    plt.title(f"Diagnosis: {label}")
    plt.axis('off')
    plt.show()

# Example usage:
annotate_image(r'E:\Dataset_chest_xray\test\Classify\p5.jpeg')
